# HMMER Profile Creation

In [ ]:
# === CONFIG: build HMMs directly from filtered folders in DIRS ===
from pathlib import Path
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import pandas as pd, numpy as np
import collections, random, re, os, shutil, subprocess, shlex
import contextlib, io
import matplotlib.pyplot as plt
import math
import seaborn as sns


# Paths
BASE        = Path("/~/HMM-PROFILES-FOR-CA-ENZYMES")
REBASE      = BASE / "OUTPUT"
LOGS        = REBASE / "logs";                  LOGS.mkdir(parents=True, exist_ok=True)
TMP         = REBASE / "tmp";                   TMP.mkdir(parents=True, exist_ok=True)
ALIGN_DIR   = REBASE / "per_class_aligned";     ALIGN_DIR.mkdir(parents=True, exist_ok=True)
SPLIT_DIR   = REBASE / "per_class_split";       SPLIT_DIR.mkdir(parents=True, exist_ok=True)
TRAIN_ALN   = REBASE / "train_aligned";         TRAIN_ALN.mkdir(parents=True, exist_ok=True)
CLEAN_ALN   = REBASE / "train_aligned_clean";   CLEAN_ALN.mkdir(parents=True, exist_ok=True)
TRIM_ALN    = REBASE / "train_aligned_trimmed"; TRIM_ALN.mkdir(parents=True, exist_ok=True)
PROFILES    = REBASE / "profiles";              PROFILES.mkdir(parents=True, exist_ok=True)
HMM_LIB     = REBASE / "hmmer_lib";             HMM_LIB.mkdir(parents=True, exist_ok=True)
RESULTS     = REBASE / "results";               RESULTS.mkdir(parents=True, exist_ok=True)
OUT     = REBASE / "clustalo_consensus_classify"

# filtered sets (edit as needed)
DIRS = [
    REBASE / "filtered_sets_per_class" / "alpha_no_change",
    REBASE / "filtered_sets_per_class" / "beta_no_change",
    REBASE / "filtered_sets_per_class" / "gamma_no_change",
    REBASE / "filtered_sets_per_class" / "delta_250_600",
    REBASE / "filtered_sets_per_class" / "zeta_150_250",
    REBASE / "filtered_sets_per_class" / "eta_300_no_max",
    REBASE / "filtered_sets_per_class" / "theta_250_550",
    REBASE / "filtered_sets_per_class" / "iota_no_change",
]

# RNG for train/test split
RNG_SEED = 42
random.seed(RNG_SEED)

def first_token(s: str) -> str:
    return (s or "").split()[0]

def write_fasta(records, path: Path):
    if records:
        SeqIO.write(records, str(path), "fasta")

# 0) Collect sequences per class from DIRS (merge & dedup by ID)
class_to_records = {}  # { "8ca.alpha.45": {id: SeqRecord, ...}, ... }
for d in DIRS:
    assert d.exists(), f"Missing folder: {d}"
    for ext in ("*.txt","*.fa","*.fasta","*.faa","*.fas"):
        for f in d.glob(ext):
            cls = f.stem
            bucket = class_to_records.setdefault(cls, {})
            for r in SeqIO.parse(str(f), "fasta"):
                rid = first_token(r.id or r.description)
                if rid not in bucket:
                    bucket[rid] = SeqRecord(Seq(str(r.seq)), id=rid, description="")

FILTERED_UPDATED_FASTAS = REBASE / "OG_from_DIRS"; FILTERED_UPDATED_FASTAS.mkdir(parents=True, exist_ok=True)
for cls, id2rec in class_to_records.items():
    out = FILTERED_UPDATED_FASTAS / f"{cls}.txt"
    write_fasta(list(id2rec.values()), out)

print(f"[OK] Wrote per-class merged FASTAs to: {FILTERED_UPDATED_FASTAS}")


In [ ]:

# Ensure Clustal omega  exists
clo = shutil.which("clustalo") or shutil.which(os.environ.get("CLUSTALO_BIN",""))
assert (clo), "Install clustalo in your env."

def run(cmd, log=None, check=False):
    print("$", cmd)
    p = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if log:
        Path(log).write_text(p.stdout + "\n--- STDERR ---\n" + p.stderr)
    if check and p.returncode != 0:
        raise RuntimeError(f"Command failed ({p.returncode})\n{p.stderr}")
    return p

# 1) Build combined unaligned FASTA & run global MSA
combined_unaln = TMP / "all_classes_DIRS.raw.faa"
seen = set()
with open(combined_unaln, "w") as outfa:
    for f in sorted(FILTERED_UPDATED_FASTAS.glob("*.txt")):
        for r in SeqIO.parse(str(f), "fasta"):
            sid = first_token(r.id or r.description)
            if sid in seen: continue
            seen.add(sid)
            r.id = sid; r.description = ""
            SeqIO.write(r, outfa, "fasta")
print(f"[combine] total unique sequences: {len(seen)}")

NEW_MSA = REBASE / "all_classes_DIRS.aln.fasta"
if clo:
    run(f'clustalo -i "{combined_unaln}" -o "{NEW_MSA}" --force --threads=4 --seqtype=Protein', 
        log=str(LOGS / "clustalo_DIRS.log"), check=True)

print("[OK] Global MSA:", NEW_MSA)

# 2) Slice per-class from global MSA
msa_records = list(SeqIO.parse(str(NEW_MSA), "fasta"))
msa_index   = { first_token(r.id or r.description): r for r in msa_records }
rows = []
for f in sorted(FILTERED_UPDATED_FASTAS.glob("*.txt")):
    cls = f.stem
    want_ids = [ first_token(r.id or r.description) for r in SeqIO.parse(str(f), "fasta") ]
    found = []
    for i in want_ids:
        if i in msa_index:
            r = msa_index[i]
            found.append(SeqRecord(Seq(str(r.seq)), id=i, description=""))
    out_aln = ALIGN_DIR / f"{cls}.aln.fasta"
    if found:
        SeqIO.write(found, str(out_aln), "fasta")
    rows.append({"class": cls, "wanted": len(want_ids), "found": len(found), "out": str(out_aln)})

slice_report = pd.DataFrame(rows).sort_values("class")
display(slice_report)
slice_report.to_csv(REBASE / "01_DIRS_slice_report.csv", index=False)

# 3) Train/Test split (80/20 stratified by class)
split_rows = []
for f in sorted(FILTERED_UPDATED_FASTAS.glob("*.txt")):
    cls = f.stem
    ids = sorted({ first_token(r.id or r.description) for r in SeqIO.parse(str(f), "fasta") })
    if len(ids) < 3:
        (SPLIT_DIR / f"{cls}.train.ids").write_text("\n".join(ids) + ("\n" if ids else ""))
        (SPLIT_DIR / f"{cls}.test.ids").write_text("")
        split_rows.append({"class":cls,"n_total":len(ids),"n_train":len(ids),"n_test":0,"note":"too_small"})
        continue
    n_train = max(2, int(0.8 * len(ids)))
    random.shuffle(ids)
    train_ids, test_ids = sorted(ids[:n_train]), sorted(ids[n_train:])
    (SPLIT_DIR / f"{cls}.train.ids").write_text("\n".join(train_ids) + "\n")
    (SPLIT_DIR / f"{cls}.test.ids").write_text("\n".join(test_ids) + "\n")
    split_rows.append({"class":cls,"n_total":len(ids),"n_train":len(train_ids),"n_test":len(test_ids),"note":""})

split_report = pd.DataFrame(split_rows).sort_values("class")
display(split_report)
split_report.to_csv(REBASE / "02_DIRS_split_report.csv", index=False)

# 4) Build TRAIN alignments by filtering per-class slices to train IDs
def strip_suffix(name: str, suffix: str) -> str:
    return name[:-len(suffix)] if name.endswith(suffix) else name

train_rows = []
for aln in sorted(ALIGN_DIR.glob("*.aln.fasta")):
    cls = strip_suffix(aln.name, ".aln.fasta")
    tid_path = SPLIT_DIR / f"{cls}.train.ids"
    if not tid_path.exists():
        print(f"[warn] missing train ids for {cls}")
        continue
    train_ids = { ln.strip() for ln in tid_path.read_text().splitlines() if ln.strip() }
    recs = []
    for r in SeqIO.parse(str(aln), "fasta"):
        sid = first_token(r.id or r.description)
        if sid in train_ids:
            recs.append(SeqRecord(Seq(str(r.seq)), id=sid, description=""))
    out = TRAIN_ALN / f"{cls}.train.aln.fasta"
    if recs:
        SeqIO.write(recs, str(out), "fasta")
    train_rows.append({"class": cls, "n_train_in_aln": len(recs), "out": str(out)})

train_report = pd.DataFrame(train_rows).sort_values("class")
display(train_report)
train_report.to_csv(REBASE / "03_DIRS_train_build_report.csv", index=False)


In [ ]:
# Ensure hmmbuild/hmmpress/hmmstat
assert shutil.which("hmmbuild"), "hmmbuild not found (hmmer_env)"
assert shutil.which("hmmpress"), "hmmpress not found (hmmer_env)"
assert shutil.which("hmmstat"),  "hmmstat not found (hmmer_env)"

def first_token(s): return s.split()[0] if s else s

valid = set("ACDEFGHIKLMNPQRSTVWYBXZ-")
def clean_strict(seq_str: str) -> str:
    s = seq_str.upper().replace(".", "-")
    return "".join(ch if ch in valid else "X" for ch in s)

def enforce_modal_length(records):
    lengths = [len(r.seq) for r in records]
    if not lengths: return [], 0, 0
    modal_len, _ = collections.Counter(lengths).most_common(1)[0]
    kept = [r for r in records if len(r.seq) == modal_len]
    return kept, modal_len, len(records) - len(kept)

# Clean to modal length
rows=[]
for aln in sorted(TRAIN_ALN.glob("*.train.aln.fasta")):
    cls = aln.stem.replace(".train.aln","")
    recs, repl = [], 0
    for r in SeqIO.parse(str(aln), "fasta"):
        raw = str(r.seq)
        cleaned = clean_strict(raw)
        repl += sum(1 for a,b in zip(raw.upper(), cleaned) if a != b)
        recs.append(SeqRecord(Seq(cleaned), id=first_token(r.id or r.description), description=""))
    kept, modal_len, dropped = enforce_modal_length(recs)
    out = CLEAN_ALN / f"{cls}.train.clean.aln.fasta"
    if kept:
        SeqIO.write(kept, str(out), "fasta")
    rows.append({"class":cls,"n_input":len(recs),"n_kept":len(kept),"dropped":dropped,"modal_len":modal_len,"replacements":repl})
pd.DataFrame(rows).sort_values("class").to_csv(REBASE / "04_DIRS_clean_report.csv", index=False)

# Trim poorly populated columns
def trim_alignment(in_fa: Path, out_fa: Path, min_symbol_frac=0.10):
    recs = list(SeqIO.parse(str(in_fa), "fasta"))
    if not recs: return False
    arr = np.array([list(str(r.seq)) for r in recs])
    gapmask = (arr == "-")
    symfrac = 1.0 - gapmask.mean(axis=0)
    keep = (symfrac >= min_symbol_frac)
    if not keep.any():
        keep = (symfrac > 0.0)
        if not keep.any(): return False
    trimmed = ["".join(row[keep]) for row in arr]
    out_recs = [SeqRecord(Seq(s), id=first_token(r.id or r.description), description="")
                for s, r in zip(trimmed, recs)]
    SeqIO.write(out_recs, str(out_fa), "fasta")
    return True

for aln in sorted(CLEAN_ALN.glob("*.train.clean.aln.fasta")):
    out = TRIM_ALN / aln.name.replace(".clean", "")
    if not out.exists():
        _ = trim_alignment(aln, out, min_symbol_frac=0.10)

# hmmbuild with fallbacks (TRIM -> CLEAN -> RAW)
def run(cmd, log=None):
    p = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if log:
        Path(log).write_text(p.stdout + "\n--- STDERR ---\n" + p.stderr)
    return p

def hmmbuild_one(cls: str) -> dict:
    p_trim  = TRIM_ALN  / f"{cls}.train.aln.fasta"
    p_clean = CLEAN_ALN / f"{cls}.train.clean.aln.fasta"
    p_raw   = TRAIN_ALN / f"{cls}.train.aln.fasta"

    candidates = [p for p in (p_trim, p_clean, p_raw) if p.exists()]
    chosen = None; chosen_n = 0; chosen_syms = 0
    for cand in candidates:
        n = sum(1 for _ in SeqIO.parse(str(cand), "fasta"))
        if n >= 2:
            chosen = cand; chosen_n = n
            chosen_syms = sum((c != '-') for r in SeqIO.parse(str(cand), "fasta") for c in str(r.seq))
            if chosen_syms > 0: break

    hmm = PROFILES / f"{cls}.hmm"
    log1= LOGS / f"{cls}.hmmbuild.log"
    log2= LOGS / f"{cls}.hmmbuild.retry.log"

    if chosen is None:     return {"class":cls,"status":"skip","note":"no_alignment_with_>=2seq"}
    if chosen_syms == 0:   return {"class":cls,"status":"skip","note":f"{chosen.name}: no non-gap symbols"}

    p1 = run(f'hmmbuild --amino --symfrac 0.2 -n "{cls}" --cpu 4 "{hmm}" "{chosen}"', log=str(log1))
    if p1.returncode == 0 and hmm.exists() and hmm.stat().st_size > 0:
        return {"class":cls,"status":"ok","note":f"{chosen.name} (symfrac=0.2)"}

    p2 = run(f'hmmbuild --amino --symfrac 0.0 -n "{cls}" --cpu 4 "{hmm}" "{chosen}"', log=str(log2))
    if p2.returncode == 0 and hmm.exists() and hmm.stat().st_size > 0:
        return {"class":cls,"status":"ok","note":f"{chosen.name} (symfrac=0.0)"}

    return {"class":cls,"status":"fail","note":f"hmmbuild_failed on {chosen.name}"}
    
with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
    subprocess.run(["hmmscan", "--help"])


slice_classes = sorted([p.stem.replace(".train.aln","") for p in TRAIN_ALN.glob("*.train.aln.fasta")])
records = [hmmbuild_one(cls) for cls in slice_classes]
hmr = pd.DataFrame(records).sort_values(["status","class"])
display(hmr)
hmr.to_csv(REBASE / "05_DIRS_hmmbuild_report.csv", index=False)

# Combine & press
combined = PROFILES / "all_classes.hmm"
if combined.exists(): combined.unlink()
parts = [p for p in sorted(PROFILES.glob("*.hmm")) if p.name != "all_classes.hmm" and p.stat().st_size > 0]
assert parts, "No non-empty HMMs built; check 05_DIRS_hmmbuild_report.csv and logs."

with open(combined, "w") as out:
    for p in parts:
        s = p.read_text()
        out.write(s if s.endswith("\n") else s + "\n")

dst = HMM_LIB / "all_classes.hmm"
shutil.copy2(combined, dst)
for ext in (".h3f",".h3i",".h3m",".h3p"):
    q = Path(str(dst) + ext)
    if q.exists(): q.unlink()
subprocess.run(["hmmpress", str(dst)], check=True, capture_output=True, text=True)


In [ ]:
print("[DONE] New pressed HMM lib:", dst)

# Verify models present
stat_out = subprocess.run(["hmmstat", str(dst)], capture_output=True, text=True, check=True).stdout
present = []
for line in stat_out.splitlines():
    if re.match(r"^\s*\d+\s+\S+", line):
        present.append(line.split()[1])
present = sorted(present)
print(f"[verify] {len(present)} models in pressed DB.")


In [ ]:
# === Held-out validation from splits ===

ALL_HMM = HMM_LIB / "all_classes.hmm"
assert ALL_HMM.exists(), f"Missing pressed HMM lib: {ALL_HMM}"

# Build FASTA of all held-out test sequences
TEST_MERGED = TMP / "dirs_heldout_test_sequences.faa"
test_map = {}  # seq_id -> true_class

for f in sorted(FILTERED_UPDATED_FASTAS.glob("*.txt")):
    cls = f.stem
    tid_path = SPLIT_DIR / f"{cls}.test.ids"
    if not tid_path.exists(): 
        continue
    want = set(x.strip() for x in tid_path.read_text().splitlines() if x.strip())
    for rec in SeqIO.parse(str(f), "fasta"):
        sid = first_token(rec.id or rec.description)
        if sid in want:
            rec.id = sid; rec.description = ""
            test_map[sid] = cls
            SeqIO.write(rec, open(TEST_MERGED, "a"), "fasta")

n_written = sum(1 for _ in SeqIO.parse(str(TEST_MERGED), "fasta")) if TEST_MERGED.exists() else 0
print(f"[held-out] wrote {n_written} sequences to {TEST_MERGED}")

# hmmscan
tbl = RESULTS / "hmmscan_dirs_heldout.tbl"
dom = RESULTS / "hmmscan_dirs_heldout.domtbl"
cmd = f'hmmscan --cpu 4 --tblout "{tbl}" --domtblout "{dom}" "{ALL_HMM}" "{TEST_MERGED}"'
print("$", cmd); subprocess.run(shlex.split(cmd), check=True)

# Parse best hits
dom_cols = ["target","tacc","tlen","query","qacc","qlen","fs_evalue","fs_score","fs_bias","num","of",
            "dom_cevalue","dom_ievalue","dom_score","dom_bias","hmmfrom","hmmto","alifrom","alito","envfrom","envto","acc","desc"]
dom_hits = pd.read_csv(dom, sep=r"\s+", comment="#", header=None, names=dom_cols, usecols=list(range(23)), engine="python")
hits = dom_hits.rename(columns={"target":"pred_model","query":"seq_id","fs_score":"bit_score","fs_evalue":"evalue"})[
    ["seq_id","pred_model","bit_score","evalue"]
]
hits["pred_class"] = hits["pred_model"].str.extract(r'^8ca\.([a-z]+)\.')

best = (hits.sort_values(["seq_id","bit_score"], ascending=[True, False])
            .groupby("seq_id", as_index=False)
            .first())

truth = pd.Series(test_map, name="true_class")
full = pd.DataFrame({"seq_id": list(test_map.keys())}).merge(best, on="seq_id", how="left")
full["pred_class"] = full["pred_class"].fillna("NO_HIT")
full = full.join(truth, on="seq_id")
full["correct"] = (full["pred_class"] == full["true_class"])

# metrics
n_eval = len(full)
n_hits = int((full["pred_class"] != "NO_HIT").sum())
coverage = n_hits / n_eval if n_eval else float("nan")
accuracy = full["correct"].mean() if n_eval else float("nan")
print(f"[held-out] evaluated={n_eval} | hits={n_hits} | coverage={coverage:.3f} | accuracy={accuracy:.3f}")

# confusion matrix
cm = pd.crosstab(full["true_class"], full["pred_class"])
display(cm)

# save
full_out = REBASE / "06_dirs_heldout_predictions.csv"
cm_out   = REBASE / "06_dirs_confusion_matrix.csv"
full.to_csv(full_out, index=False); cm.to_csv(cm_out)
print("[OK] wrote", full_out, cm_out)




In [ ]:
UNIPROT_BIG = BASE / "uniprotkb_carbonic_anhydrase_2025_08_22.fasta"
ALL_HMM     = HMM_LIB / "all_classes.hmm"
assert UNIPROT_BIG.exists(), f"Missing UniProt FASTA: {UNIPROT_BIG}"
assert ALL_HMM.exists(),     f"Missing HMM DB: {ALL_HMM}"

# 10k

In [ ]:
# === Classify first 10k UniProt sequences with the new HMM lib; write CSV with confidence ===

N_TAKE          = 10_000
CPU             = 4
MIN_BITSCORE    = None   
MAX_EVALUE      = None   

def first_token(s: str) -> str:
    return s.split()[0] if s else s

# 1) Subset
subset_fa = TMP / f"uniprot_first_{N_TAKE}.faa"
with open(subset_fa, "w") as out:
    for i, rec in enumerate(SeqIO.parse(str(UNIPROT_BIG), "fasta"), start=1):
        rec.id = first_token(rec.id or rec.description); rec.description = ""
        SeqIO.write(rec, out, "fasta")
        if i >= N_TAKE: break

# 2) hmmscan
tbl = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.tbl"
dom = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.domtbl"
log = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.log"
cmd = f'hmmscan --cpu {CPU} --noali --notextw --tblout "{tbl}" --domtblout "{dom}" "{ALL_HMM}" "{subset_fa}"'
with open(log, "w") as lf:
    subprocess.run(shlex.split(cmd), check=True, stdout=lf, stderr=lf)

# 3) parse, best hit per seq, apply thresholds, compute confidence
dom_cols = ["target","tacc","tlen","query","qacc","qlen","fs_evalue","fs_score","fs_bias","num","of",
            "dom_cevalue","dom_ievalue","dom_score","dom_bias","hmmfrom","hmmto","alifrom","alito","envfrom","envto","acc","desc"]
dom_hits = pd.read_csv(dom, sep=r"\s+", comment="#", header=None, names=dom_cols, usecols=list(range(23)), engine="python")

hits = dom_hits.rename(columns={
    "target":"pred_model", "query":"seq_id", "fs_score":"bit_score", "fs_evalue":"evalue"
})[["seq_id","pred_model","bit_score","evalue"]]

# collapse model -> class (e.g., 8ca.theta.45 -> theta)
hits["pred_class"] = hits["pred_model"].str.extract(r'^8ca\.([a-z]+)\.')

# best by bit score
best = (hits.sort_values(["seq_id","bit_score"], ascending=[True, False])
             .groupby("seq_id", as_index=False)
             .first())

# add NO_HITs & thresholds
subset_ids = [r.id for r in SeqIO.parse(str(subset_fa), "fasta")]
lab = pd.DataFrame({"seq_id": subset_ids}).merge(best, on="seq_id", how="left")
lab["pred_class"] = lab["pred_class"].fillna("NO_HIT")

# Optional thresholds -> demote to NO_HIT
if MIN_BITSCORE is not None:
    lab.loc[lab["bit_score"].notna() & (lab["bit_score"] < MIN_BITSCORE), "pred_class"] = "NO_HIT"
if MAX_EVALUE is not None:
    lab.loc[lab["evalue"].notna() & (lab["evalue"] > MAX_EVALUE), "pred_class"] = "NO_HIT"

# Confidence score: -log10(evalue) (higher is more confident); NaN for NO_HIT/empty
def conf_from_evalue(ev):
    try:
        if pd.isna(ev): return np.nan
        evf = float(ev)
        if evf <= 0.0: return 300.0  # cap tiny e-values
        return -np.log10(evf)
    except Exception:
        return np.nan

lab["confidence"] = lab["evalue"].apply(conf_from_evalue)

# Save detailed CSV
out_csv = RESULTS / f"uniprot_first_{N_TAKE}_labels_with_confidence.csv"
lab.to_csv(out_csv, index=False)
print(f"[OK] labeled {len(lab):,} seqs → {out_csv}")


In [14]:
# Optional: distribution table & bar plot

counts = (lab["pred_class"].value_counts(dropna=False)
             .rename_axis("class")
             .reset_index(name="count")
             .sort_values("count", ascending=False))
display(counts)

plt.figure(figsize=(10,5))
plt.bar(counts["class"], counts["count"])
plt.title("Distribution of HMMER-predicted Classes (first 10k sequences)")
plt.xlabel("Predicted Class"); plt.ylabel("Count")
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

counts.to_csv(RESULTS / f"uniprot_first_{N_TAKE}_label_distribution.csv", index=False)


In [15]:
# === Load labeled 10k results & ensure 'confidence' exists ===

LABELS  = RESULTS / "uniprot_first_10000_labels_with_confidence.csv"

df = pd.read_csv(LABELS)
print(f"[load] {len(df):,} sequences")

# If confidence is missing, compute from e-value: confidence = -log10(evalue)
if "confidence" not in df.columns:
    def conf_from_evalue(ev):
        try:
            if pd.isna(ev): return np.nan
            evf = float(ev)
            if evf <= 0.0: return 300.0  # cap tiny e-values
            return -np.log10(evf)
        except Exception:
            return np.nan
    df["confidence"] = df["evalue"].apply(conf_from_evalue)

# Make a filtered frame with actual hits only
hits = df[df["pred_class"].ne("NO_HIT")].copy()
print(f"[hits] {len(hits):,} with assigned class (exclude NO_HIT)")


In [16]:
# === Overall confidence distribution: histogram + ECDF ===

conf = hits["confidence"].dropna().values
if conf.size == 0:
    print("No confidences to plot.")
else:
    # Histogram
    plt.figure(figsize=(8,4))
    # Choose bins up to a reasonable cap (ignore extreme 300 cap if present)
    finite = conf[np.isfinite(conf)]
    xmax = np.percentile(finite, 99.5) if finite.size else 10
    bins = np.linspace(0, xmax, 40)
    plt.hist(conf, bins=bins)
    plt.title("Confidence (−log10 e-value) — Overall")
    plt.xlabel("Confidence (higher = more confident)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_hist.png", dpi=150)
    plt.show()

    # ECDF
    def ecdf(vals):
        x = np.sort(vals)
        y = (np.arange(1, len(x)+1) / len(x)) * 100.0
        return x, y

    x, y = ecdf(conf)
    plt.figure(figsize=(8,4))
    plt.step(x, y, where="post")
    plt.title("Confidence ECDF — Overall")
    plt.xlabel("Confidence (−log10 e-value)")
    plt.ylabel("Percent ≤ confidence")
    plt.grid(True, alpha=0.3, linestyle=":")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_ecdf.png", dpi=150)
    plt.show()


E-value (expected value) is the expected number of false matches you’d see with a score ≥ this one by chance, given the database size.

In [ ]:
# === Confidence by predicted class: boxplots for top-N classes ===

TOP_N = 10  

# Count per class and pick top-N
counts = (hits["pred_class"]
          .value_counts()
          .sort_values(ascending=False))
top_classes = counts.head(TOP_N).index.tolist()

# Prepare data for boxplot
data = [hits.loc[hits["pred_class"]==c, "confidence"].dropna().values for c in top_classes]

plt.figure(figsize=(max(10, 1.0*len(top_classes)+4), 5))
bp = plt.boxplot(data, labels=top_classes, showfliers=False)
plt.title(f"Confidence by Predicted Class (top {len(top_classes)} by count)")
plt.ylabel("Confidence (−log10 e-value)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_boxplot_topN.png", dpi=150)
plt.show()


In [18]:
# === Per-class confidence summary + bar chart of means ===

summary = (
    hits.dropna(subset=["confidence"])
        .groupby("pred_class")["confidence"]
        .agg(n="count", mean="mean", median="median",
             q25=lambda s: np.percentile(s, 25),
             q75=lambda s: np.percentile(s, 75))
        .sort_values("mean", ascending=False)
        .reset_index()
)

summary_path = RESULTS / "conf_per_class_summary.csv"
summary.to_csv(summary_path, index=False)
print("[OK] wrote", summary_path)
display(summary.head(20))

# Bar of mean confidence for top-M by count (to avoid tiny classes dominating)
M = 12
top_by_count = (hits["pred_class"].value_counts().head(M).index)
summ_plot = summary[summary["pred_class"].isin(top_by_count)].copy()

plt.figure(figsize=(10,5))
plt.bar(summ_plot["pred_class"], summ_plot["mean"])
plt.title("Mean Confidence by Class (top by count)")
plt.ylabel("Mean confidence (−log10 e-value)")
plt.xlabel("Predicted class")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_mean_bar_topCount.png", dpi=150)
plt.show()


# 50k

In [ ]:
# === Classify first 10k UniProt sequences with the new HMM lib; write CSV with confidence ===

N_TAKE          = 50_000
CPU             = 4
MIN_BITSCORE    = None   
MAX_EVALUE      = None  

def first_token(s: str) -> str:
    return s.split()[0] if s else s

# 1) Subset
subset_fa = TMP / f"uniprot_first_{N_TAKE}.faa"
with open(subset_fa, "w") as out:
    for i, rec in enumerate(SeqIO.parse(str(UNIPROT_BIG), "fasta"), start=1):
        rec.id = first_token(rec.id or rec.description); rec.description = ""
        SeqIO.write(rec, out, "fasta")
        if i >= N_TAKE: break

# 2) hmmscan
tbl = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.tbl"
dom = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.domtbl"
log = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.log"
cmd = f'hmmscan --cpu {CPU} --noali --notextw --tblout "{tbl}" --domtblout "{dom}" "{ALL_HMM}" "{subset_fa}"'
with open(log, "w") as lf:
    subprocess.run(shlex.split(cmd), check=True, stdout=lf, stderr=lf)

# 3) parse, best hit per seq, apply thresholds, compute confidence
dom_cols = ["target","tacc","tlen","query","qacc","qlen","fs_evalue","fs_score","fs_bias","num","of",
            "dom_cevalue","dom_ievalue","dom_score","dom_bias","hmmfrom","hmmto","alifrom","alito","envfrom","envto","acc","desc"]
dom_hits = pd.read_csv(dom, sep=r"\s+", comment="#", header=None, names=dom_cols, usecols=list(range(23)), engine="python")

hits = dom_hits.rename(columns={
    "target":"pred_model", "query":"seq_id", "fs_score":"bit_score", "fs_evalue":"evalue"
})[["seq_id","pred_model","bit_score","evalue"]]

# collapse model -> class (e.g., 8ca.theta.45 -> theta)
hits["pred_class"] = hits["pred_model"].str.extract(r'^8ca\.([a-z]+)\.')

# best by bit score
best = (hits.sort_values(["seq_id","bit_score"], ascending=[True, False])
             .groupby("seq_id", as_index=False)
             .first())

# add NO_HITs & thresholds
subset_ids = [r.id for r in SeqIO.parse(str(subset_fa), "fasta")]
lab = pd.DataFrame({"seq_id": subset_ids}).merge(best, on="seq_id", how="left")
lab["pred_class"] = lab["pred_class"].fillna("NO_HIT")

# Optional thresholds -> demote to NO_HIT
if MIN_BITSCORE is not None:
    lab.loc[lab["bit_score"].notna() & (lab["bit_score"] < MIN_BITSCORE), "pred_class"] = "NO_HIT"
if MAX_EVALUE is not None:
    lab.loc[lab["evalue"].notna() & (lab["evalue"] > MAX_EVALUE), "pred_class"] = "NO_HIT"

# Confidence score: -log10(evalue) (higher is more confident); NaN for NO_HIT/empty
def conf_from_evalue(ev):
    try:
        if pd.isna(ev): return np.nan
        evf = float(ev)
        if evf <= 0.0: return 300.0  # cap tiny e-values
        return -np.log10(evf)
    except Exception:
        return np.nan

lab["confidence"] = lab["evalue"].apply(conf_from_evalue)

# Save detailed CSV
out_csv = RESULTS / f"uniprot_first_{N_TAKE}_labels_with_confidence.csv"
lab.to_csv(out_csv, index=False)
print(f"[OK] labeled {len(lab):,} seqs → {out_csv}")


In [26]:
# Optional: distribution table & bar plot
import matplotlib.pyplot as plt

counts = (lab["pred_class"].value_counts(dropna=False)
             .rename_axis("class")
             .reset_index(name="count")
             .sort_values("count", ascending=False))
display(counts)

plt.figure(figsize=(10,5))
plt.bar(counts["class"], counts["count"])
plt.title("Distribution of HMMER-predicted Classes (first 50k sequences)")
plt.xlabel("Predicted Class"); plt.ylabel("Count")
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

counts.to_csv(RESULTS / f"uniprot_first_{N_TAKE}_label_distribution.csv", index=False)


In [21]:
# === Load labeled 10k results & ensure 'confidence' exists ===
from pathlib import Path
import pandas as pd
import numpy as np

BASE    = Path("/mnt/c/Users/SAM/CODE/HMMR")
REBASE  = BASE / "HMMR_RE"
RESULTS = REBASE / "results"
LABELS  = RESULTS / "uniprot_first_50000_labels_with_confidence.csv"

df = pd.read_csv(LABELS)
print(f"[load] {len(df):,} sequences")

# If confidence is missing, compute from e-value: confidence = -log10(evalue)
if "confidence" not in df.columns:
    def conf_from_evalue(ev):
        try:
            if pd.isna(ev): return np.nan
            evf = float(ev)
            if evf <= 0.0: return 300.0  # cap tiny e-values
            return -np.log10(evf)
        except Exception:
            return np.nan
    df["confidence"] = df["evalue"].apply(conf_from_evalue)

# Make a filtered frame with actual hits only
hits = df[df["pred_class"].ne("NO_HIT")].copy()
print(f"[hits] {len(hits):,} with assigned class (exclude NO_HIT)")


In [ ]:
# === Overall confidence distribution: histogram + ECDF ===
import numpy as np
import matplotlib.pyplot as plt

conf = hits["confidence"].dropna().values
if conf.size == 0:
    print("No confidences to plot.")
else:
    # Histogram
    plt.figure(figsize=(8,4))
    finite = conf[np.isfinite(conf)]
    xmax = np.percentile(finite, 99.5) if finite.size else 10
    bins = np.linspace(0, xmax, 40)
    plt.hist(conf, bins=bins)
    plt.title("Confidence (−log10 e-value) — Overall")
    plt.xlabel("Confidence (higher = more confident)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_hist.png", dpi=150)
    plt.show()

    # ECDF
    def ecdf(vals):
        x = np.sort(vals)
        y = (np.arange(1, len(x)+1) / len(x)) * 100.0
        return x, y

    x, y = ecdf(conf)
    plt.figure(figsize=(8,4))
    plt.step(x, y, where="post")
    plt.title("Confidence ECDF — Overall")
    plt.xlabel("Confidence (−log10 e-value)")
    plt.ylabel("Percent ≤ confidence")
    plt.grid(True, alpha=0.3, linestyle=":")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_ecdf.png", dpi=150)
    plt.show()


In [ ]:
# === Confidence by predicted class: boxplots for top-N classes ===
import numpy as np
import matplotlib.pyplot as plt

TOP_N = 10 

# Count per class and pick top-N
counts = (hits["pred_class"]
          .value_counts()
          .sort_values(ascending=False))
top_classes = counts.head(TOP_N).index.tolist()

# Prepare data for boxplot
data = [hits.loc[hits["pred_class"]==c, "confidence"].dropna().values for c in top_classes]

plt.figure(figsize=(max(10, 1.0*len(top_classes)+4), 5))
bp = plt.boxplot(data, labels=top_classes, showfliers=False)
plt.title(f"Confidence by Predicted Class (top {len(top_classes)} by count)")
plt.ylabel("Confidence (−log10 e-value)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_boxplot_topN.png", dpi=150)
plt.show()


In [24]:
# === Per-class confidence summary + bar chart of means ===
import numpy as np
import matplotlib.pyplot as plt

summary = (
    hits.dropna(subset=["confidence"])
        .groupby("pred_class")["confidence"]
        .agg(n="count", mean="mean", median="median",
             q25=lambda s: np.percentile(s, 25),
             q75=lambda s: np.percentile(s, 75))
        .sort_values("mean", ascending=False)
        .reset_index()
)

summary_path = RESULTS / "conf_per_class_summary.csv"
summary.to_csv(summary_path, index=False)
print("[OK] wrote", summary_path)
display(summary.head(20))

# Bar of mean confidence for top-M by count (to avoid tiny classes dominating)
M = 12
top_by_count = (hits["pred_class"].value_counts().head(M).index)
summ_plot = summary[summary["pred_class"].isin(top_by_count)].copy()

plt.figure(figsize=(10,5))
plt.bar(summ_plot["pred_class"], summ_plot["mean"])
plt.title("Mean Confidence by Class (top by count)")
plt.ylabel("Mean confidence (−log10 e-value)")
plt.xlabel("Predicted class")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_mean_bar_topCount.png", dpi=150)
plt.show()


# 100k

In [ ]:
os.cpu_count()

In [ ]:
# === Classify first 10k UniProt sequences with the new HMM lib; write CSV with confidence ===

N_TAKE          = 100_000
CPU             = 8
MIN_BITSCORE    = None   # e.g., 25.0 (optional)
MAX_EVALUE      = None   # e.g., 1e-5 (optional)

def first_token(s: str) -> str:
    return s.split()[0] if s else s

# 1) Subset
subset_fa = TMP / f"uniprot_first_{N_TAKE}.faa"
with open(subset_fa, "w") as out:
    for i, rec in enumerate(SeqIO.parse(str(UNIPROT_BIG), "fasta"), start=1):
        rec.id = first_token(rec.id or rec.description); rec.description = ""
        SeqIO.write(rec, out, "fasta")
        if i >= N_TAKE: break

# 2) hmmscan
tbl = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.tbl"
dom = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.domtbl"
log = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.log"
cmd = f'hmmscan --cpu {CPU} --noali --notextw --tblout "{tbl}" --domtblout "{dom}" "{ALL_HMM}" "{subset_fa}"'
with open(log, "w") as lf:
    subprocess.run(shlex.split(cmd), check=True, stdout=lf, stderr=lf)

# 3) parse, best hit per seq, apply thresholds, compute confidence
dom_cols = ["target","tacc","tlen","query","qacc","qlen","fs_evalue","fs_score","fs_bias","num","of",
            "dom_cevalue","dom_ievalue","dom_score","dom_bias","hmmfrom","hmmto","alifrom","alito","envfrom","envto","acc","desc"]
dom_hits = pd.read_csv(dom, sep=r"\s+", comment="#", header=None, names=dom_cols, usecols=list(range(23)), engine="python")

hits = dom_hits.rename(columns={
    "target":"pred_model", "query":"seq_id", "fs_score":"bit_score", "fs_evalue":"evalue"
})[["seq_id","pred_model","bit_score","evalue"]]

# collapse model -> class (e.g., 8ca.theta.45 -> theta)
hits["pred_class"] = hits["pred_model"].str.extract(r'^8ca\.([a-z]+)\.')

# best by bit score
best = (hits.sort_values(["seq_id","bit_score"], ascending=[True, False])
             .groupby("seq_id", as_index=False)
             .first())

# add NO_HITs & thresholds
subset_ids = [r.id for r in SeqIO.parse(str(subset_fa), "fasta")]
lab = pd.DataFrame({"seq_id": subset_ids}).merge(best, on="seq_id", how="left")
lab["pred_class"] = lab["pred_class"].fillna("NO_HIT")

# Optional thresholds -> demote to NO_HIT
if MIN_BITSCORE is not None:
    lab.loc[lab["bit_score"].notna() & (lab["bit_score"] < MIN_BITSCORE), "pred_class"] = "NO_HIT"
if MAX_EVALUE is not None:
    lab.loc[lab["evalue"].notna() & (lab["evalue"] > MAX_EVALUE), "pred_class"] = "NO_HIT"

# Confidence score: -log10(evalue) (higher is more confident); NaN for NO_HIT/empty
def conf_from_evalue(ev):
    try:
        if pd.isna(ev): return np.nan
        evf = float(ev)
        if evf <= 0.0: return 300.0  # cap tiny e-values
        return -np.log10(evf)
    except Exception:
        return np.nan

lab["confidence"] = lab["evalue"].apply(conf_from_evalue)

# Save detailed CSV
out_csv = RESULTS / f"uniprot_first_{N_TAKE}_labels_with_confidence.csv"
lab.to_csv(out_csv, index=False)
print(f"[OK] labeled {len(lab):,} seqs → {out_csv}")


In [28]:
# Optional: distribution table & bar plot
import matplotlib.pyplot as plt

counts = (lab["pred_class"].value_counts(dropna=False)
             .rename_axis("class")
             .reset_index(name="count")
             .sort_values("count", ascending=False))
display(counts)

plt.figure(figsize=(10,5))
plt.bar(counts["class"], counts["count"])
plt.title("Distribution of HMMER-predicted Classes (first 100k sequences)")
plt.xlabel("Predicted Class"); plt.ylabel("Count")
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

counts.to_csv(RESULTS / f"uniprot_first_{N_TAKE}_label_distribution.csv", index=False)


In [29]:
# === Load labeled 10k results & ensure 'confidence' exists ===
from pathlib import Path
import pandas as pd
import numpy as np

BASE    = Path("/mnt/c/Users/SAM/CODE/HMMR")
REBASE  = BASE / "HMMR_RE"
RESULTS = REBASE / "results"
LABELS  = RESULTS / "uniprot_first_100000_labels_with_confidence.csv"

df = pd.read_csv(LABELS)
print(f"[load] {len(df):,} sequences")

# If confidence is missing, compute from e-value: confidence = -log10(evalue)
if "confidence" not in df.columns:
    def conf_from_evalue(ev):
        try:
            if pd.isna(ev): return np.nan
            evf = float(ev)
            if evf <= 0.0: return 300.0  # cap tiny e-values
            return -np.log10(evf)
        except Exception:
            return np.nan
    df["confidence"] = df["evalue"].apply(conf_from_evalue)

# Make a filtered frame with actual hits only
hits = df[df["pred_class"].ne("NO_HIT")].copy()
print(f"[hits] {len(hits):,} with assigned class (exclude NO_HIT)")


In [ ]:
# === Overall confidence distribution: histogram + ECDF ===
import numpy as np
import matplotlib.pyplot as plt

conf = hits["confidence"].dropna().values
if conf.size == 0:
    print("No confidences to plot.")
else:
    # Histogram
    plt.figure(figsize=(8,4))
    finite = conf[np.isfinite(conf)]
    xmax = np.percentile(finite, 99.5) if finite.size else 10
    bins = np.linspace(0, xmax, 40)
    plt.hist(conf, bins=bins)
    plt.title("Confidence (−log10 e-value) — Overall")
    plt.xlabel("Confidence (higher = more confident)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_hist.png", dpi=150)
    plt.show()

    # ECDF
    def ecdf(vals):
        x = np.sort(vals)
        y = (np.arange(1, len(x)+1) / len(x)) * 100.0
        return x, y

    x, y = ecdf(conf)
    plt.figure(figsize=(8,4))
    plt.step(x, y, where="post")
    plt.title("Confidence ECDF — Overall")
    plt.xlabel("Confidence (−log10 e-value)")
    plt.ylabel("Percent ≤ confidence")
    plt.grid(True, alpha=0.3, linestyle=":")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_ecdf.png", dpi=150)
    plt.show()


In [32]:
# === Confidence by predicted class: boxplots for top-N classes ===
import numpy as np
import matplotlib.pyplot as plt

TOP_N = 10  # change if you want more/less classes in the figure

# Count per class and pick top-N
counts = (hits["pred_class"]
          .value_counts()
          .sort_values(ascending=False))
top_classes = counts.head(TOP_N).index.tolist()

# Prepare data for boxplot
data = [hits.loc[hits["pred_class"]==c, "confidence"].dropna().values for c in top_classes]

plt.figure(figsize=(max(10, 1.0*len(top_classes)+4), 5))
bp = plt.boxplot(data, labels=top_classes, showfliers=False)
plt.title(f"Confidence by Predicted Class (top {len(top_classes)} by count)")
plt.ylabel("Confidence (−log10 e-value)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_boxplot_topN.png", dpi=150)
plt.show()


In [31]:
# === Per-class confidence summary + bar chart of means ===
import numpy as np
import matplotlib.pyplot as plt

summary = (
    hits.dropna(subset=["confidence"])
        .groupby("pred_class")["confidence"]
        .agg(n="count", mean="mean", median="median",
             q25=lambda s: np.percentile(s, 25),
             q75=lambda s: np.percentile(s, 75))
        .sort_values("mean", ascending=False)
        .reset_index()
)

summary_path = RESULTS / "conf_per_class_summary.csv"
summary.to_csv(summary_path, index=False)
print("[OK] wrote", summary_path)
display(summary.head(20))

# Bar of mean confidence for top-M by count (to avoid tiny classes dominating)
M = 12
top_by_count = (hits["pred_class"].value_counts().head(M).index)
summ_plot = summary[summary["pred_class"].isin(top_by_count)].copy()

plt.figure(figsize=(10,5))
plt.bar(summ_plot["pred_class"], summ_plot["mean"])
plt.title("Mean Confidence by Class (top by count)")
plt.ylabel("Mean confidence (−log10 e-value)")
plt.xlabel("Predicted class")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_mean_bar_topCount.png", dpi=150)
plt.show()


# 150k

In [ ]:
# === Classify first 150k UniProt sequences with the new HMM lib; write CSV with confidence ===


N_TAKE          = 150_000
CPU             = 8
MIN_BITSCORE    = None   # e.g., 25.0 (optional)
MAX_EVALUE      = None   # e.g., 1e-5 (optional)

def first_token(s: str) -> str:
    return s.split()[0] if s else s

# 1) Subset
subset_fa = TMP / f"uniprot_first_{N_TAKE}.faa"
with open(subset_fa, "w") as out:
    for i, rec in enumerate(SeqIO.parse(str(UNIPROT_BIG), "fasta"), start=1):
        rec.id = first_token(rec.id or rec.description); rec.description = ""
        SeqIO.write(rec, out, "fasta")
        if i >= N_TAKE: break

# 2) hmmscan
tbl = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.tbl"
dom = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.domtbl"
log = RESULTS / f"hmmscan_uniprot_first_{N_TAKE}.log"
cmd = f'hmmscan --cpu {CPU} --noali --notextw --tblout "{tbl}" --domtblout "{dom}" "{ALL_HMM}" "{subset_fa}"'
with open(log, "w") as lf:
    subprocess.run(shlex.split(cmd), check=True, stdout=lf, stderr=lf)

# 3) parse, best hit per seq, apply thresholds, compute confidence
dom_cols = ["target","tacc","tlen","query","qacc","qlen","fs_evalue","fs_score","fs_bias","num","of",
            "dom_cevalue","dom_ievalue","dom_score","dom_bias","hmmfrom","hmmto","alifrom","alito","envfrom","envto","acc","desc"]
dom_hits = pd.read_csv(dom, sep=r"\s+", comment="#", header=None, names=dom_cols, usecols=list(range(23)), engine="python")

hits = dom_hits.rename(columns={
    "target":"pred_model", "query":"seq_id", "fs_score":"bit_score", "fs_evalue":"evalue"
})[["seq_id","pred_model","bit_score","evalue"]]

# collapse model -> class (e.g., 8ca.theta.45 -> theta)
hits["pred_class"] = hits["pred_model"].str.extract(r'^8ca\.([a-z]+)\.')

# best by bit score
best = (hits.sort_values(["seq_id","bit_score"], ascending=[True, False])
             .groupby("seq_id", as_index=False)
             .first())

# add NO_HITs & thresholds
subset_ids = [r.id for r in SeqIO.parse(str(subset_fa), "fasta")]
lab = pd.DataFrame({"seq_id": subset_ids}).merge(best, on="seq_id", how="left")
lab["pred_class"] = lab["pred_class"].fillna("NO_HIT")

# Optional thresholds -> demote to NO_HIT
if MIN_BITSCORE is not None:
    lab.loc[lab["bit_score"].notna() & (lab["bit_score"] < MIN_BITSCORE), "pred_class"] = "NO_HIT"
if MAX_EVALUE is not None:
    lab.loc[lab["evalue"].notna() & (lab["evalue"] > MAX_EVALUE), "pred_class"] = "NO_HIT"

# Confidence score: -log10(evalue) (higher is more confident); NaN for NO_HIT/empty
def conf_from_evalue(ev):
    try:
        if pd.isna(ev): return np.nan
        evf = float(ev)
        if evf <= 0.0: return 300.0  # cap tiny e-values
        return -np.log10(evf)
    except Exception:
        return np.nan

lab["confidence"] = lab["evalue"].apply(conf_from_evalue)

# Save detailed CSV
out_csv = RESULTS / f"uniprot_first_{N_TAKE}_labels_with_confidence.csv"
lab.to_csv(out_csv, index=False)
print(f"[OK] labeled {len(lab):,} seqs → {out_csv}")


In [37]:
# Optional: distribution table & bar plot
import matplotlib.pyplot as plt

counts = (lab["pred_class"].value_counts(dropna=False)
             .rename_axis("class")
             .reset_index(name="count")
             .sort_values("count", ascending=False))
display(counts)

plt.figure(figsize=(10,5))
plt.bar(counts["class"], counts["count"])
plt.title("Distribution of HMMER-predicted Classes (first 150k sequences)")
plt.xlabel("Predicted Class"); plt.ylabel("Count")
plt.xticks(rotation=45, ha='right'); plt.tight_layout(); plt.show()

counts.to_csv(RESULTS / f"uniprot_first_{N_TAKE}_label_distribution.csv", index=False)


In [38]:
# === Load labeled 10k results & ensure 'confidence' exists ===
from pathlib import Path
import pandas as pd
import numpy as np

BASE    = Path("/mnt/c/Users/SAM/CODE/HMMR")
REBASE  = BASE / "HMMR_RE"
RESULTS = REBASE / "results"
LABELS  = RESULTS / "uniprot_first_150000_labels_with_confidence.csv"

df = pd.read_csv(LABELS)
print(f"[load] {len(df):,} sequences")

# If confidence is missing, compute from e-value: confidence = -log10(evalue)
if "confidence" not in df.columns:
    def conf_from_evalue(ev):
        try:
            if pd.isna(ev): return np.nan
            evf = float(ev)
            if evf <= 0.0: return 300.0  # cap tiny e-values
            return -np.log10(evf)
        except Exception:
            return np.nan
    df["confidence"] = df["evalue"].apply(conf_from_evalue)

# Make a filtered frame with actual hits only
hits = df[df["pred_class"].ne("NO_HIT")].copy()
print(f"[hits] {len(hits):,} with assigned class (exclude NO_HIT)")


In [ ]:
# === Overall confidence distribution: histogram + ECDF ===
import numpy as np
import matplotlib.pyplot as plt

conf = hits["confidence"].dropna().values
if conf.size == 0:
    print("No confidences to plot.")
else:
    # Histogram
    plt.figure(figsize=(8,4))

    finite = conf[np.isfinite(conf)]
    xmax = np.percentile(finite, 99.5) if finite.size else 10
    bins = np.linspace(0, xmax, 40)
    plt.hist(conf, bins=bins)
    plt.title("Confidence (−log10 e-value) — Overall")
    plt.xlabel("Confidence (higher = more confident)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_hist.png", dpi=150)
    plt.show()

    # ECDF
    def ecdf(vals):
        x = np.sort(vals)
        y = (np.arange(1, len(x)+1) / len(x)) * 100.0
        return x, y

    x, y = ecdf(conf)
    plt.figure(figsize=(8,4))
    plt.step(x, y, where="post")
    plt.title("Confidence ECDF — Overall")
    plt.xlabel("Confidence (−log10 e-value)")
    plt.ylabel("Percent ≤ confidence")
    plt.grid(True, alpha=0.3, linestyle=":")
    plt.tight_layout()
    plt.savefig(RESULTS / "conf_overall_ecdf.png", dpi=150)
    plt.show()


In [ ]:
# === Confidence by predicted class: boxplots for top-N classes ===
import numpy as np
import matplotlib.pyplot as plt

TOP_N = 10  

# Count per class and pick top-N
counts = (hits["pred_class"]
          .value_counts()
          .sort_values(ascending=False))
top_classes = counts.head(TOP_N).index.tolist()

# Prepare data for boxplot
data = [hits.loc[hits["pred_class"]==c, "confidence"].dropna().values for c in top_classes]

plt.figure(figsize=(max(10, 1.0*len(top_classes)+4), 5))
bp = plt.boxplot(data, labels=top_classes, showfliers=False)
plt.title(f"Confidence by Predicted Class (top {len(top_classes)} by count)")
plt.ylabel("Confidence (−log10 e-value)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_boxplot_topN.png", dpi=150)
plt.show()


In [41]:
# === Per-class confidence summary + bar chart of means ===
import numpy as np
import matplotlib.pyplot as plt

summary = (
    hits.dropna(subset=["confidence"])
        .groupby("pred_class")["confidence"]
        .agg(n="count", mean="mean", median="median",
             q25=lambda s: np.percentile(s, 25),
             q75=lambda s: np.percentile(s, 75))
        .sort_values("mean", ascending=False)
        .reset_index()
)

summary_path = RESULTS / "conf_per_class_summary.csv"
summary.to_csv(summary_path, index=False)
print("[OK] wrote", summary_path)
display(summary.head(20))

# Bar of mean confidence for top-M by count (to avoid tiny classes dominating)
M = 12
top_by_count = (hits["pred_class"].value_counts().head(M).index)
summ_plot = summary[summary["pred_class"].isin(top_by_count)].copy()

plt.figure(figsize=(10,5))
plt.bar(summ_plot["pred_class"], summ_plot["mean"])
plt.title("Mean Confidence by Class (top by count)")
plt.ylabel("Mean confidence (−log10 e-value)")
plt.xlabel("Predicted class")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(RESULTS / "conf_by_class_mean_bar_topCount.png", dpi=150)
plt.show()


# Consensus w/ Clustal O

In [ ]:
# === Sanity check: Did training sequences appear in the labeling CSV, and were they labeled correctly? ===


# 1) Point to the labeling CSV 
LABELS_CSV = RESULTS / "uniprot_first_150000_labels_with_confidence.csv"   

# ---------------- Helpers ----------------
def first_token(s: str) -> str:
    return (s or "").split()[0]

def extract_simple_class(model_or_class: str) -> str:
    """
    Normalize a model or class string to the simple class name.
    Examples:
      '8ca.alpha.45' -> 'alpha'
      'alpha'        -> 'alpha'
    """
    if pd.isna(model_or_class):
        return None
    m = re.match(r'^8ca\.([a-z]+)\.', str(model_or_class))
    return m.group(1) if m else str(model_or_class)

# ---------------- Load labeled CSV ----------------
assert LABELS_CSV.exists(), f"Labels CSV not found: {LABELS_CSV}"
df = pd.read_csv(LABELS_CSV)
# Ensure expected columns exist
for col in ["seq_id", "pred_class", "pred_model", "bit_score", "evalue", "confidence"]:
    if col not in df.columns:
        # tolerate older CSVs (without confidence)
        if col == "confidence":
            df["confidence"] = np.nan
        else:
            raise ValueError(f"Expected column '{col}' not found in {LABELS_CSV}")

# Normalize predicted class to simple form
df["pred_class_simple"] = df["pred_class"].apply(extract_simple_class)

# ---------------- Build training ID → class map from DIRS ----------------
train_id_to_class = {}   # seq_id -> simple_class
train_dupe_ids = set()   # track if an ID appears in more than one class (shouldn't)

for d in DIRS:
    d = Path(d)
    assert d.exists(), f"Missing training folder: {d}"
    for ext in ("*.txt", "*.fa", "*.fasta", "*.faa", "*.fas"):
        for fp in d.glob(ext):
            # Class name from filename stem, normalize to simple class
            cls_simple = extract_simple_class(fp.stem)
            for r in SeqIO.parse(str(fp), "fasta"):
                rid = first_token(r.id or r.description)
                if rid in train_id_to_class and train_id_to_class[rid] != cls_simple:
                    train_dupe_ids.add(rid)
                train_id_to_class.setdefault(rid, cls_simple)

if train_dupe_ids:
    print(f"[warn] {len(train_dupe_ids)} training IDs occurred under multiple classes. Showing a few:",
          sorted(list(train_dupe_ids))[:5])

# ---------------- Intersect training IDs with labeled CSV ----------------
csv_ids = set(df["seq_id"].astype(str))
train_ids = set(train_id_to_class.keys())
overlap_ids = csv_ids & train_ids

print(f"[info] Labeled CSV rows: {len(df):,}")
print(f"[info] Unique seq_ids in CSV: {len(csv_ids):,}")
print(f"[info] Training IDs total: {len(train_ids):,}")
print(f"[info] Overlap (training IDs found in CSV): {len(overlap_ids):,}")

if len(overlap_ids) == 0:
    print("No training sequences found in the labeled CSV. (This is expected, UniProt set should not contain training IDs.)")
else:
    # Build a frame of overlaps with truth vs prediction
    truth = pd.DataFrame({
        "seq_id": list(overlap_ids),
        "true_class_simple": [train_id_to_class[sid] for sid in overlap_ids]
    })
    merged = truth.merge(
        df[["seq_id", "pred_class_simple", "pred_class", "pred_model", "bit_score", "evalue", "confidence"]],
        on="seq_id", how="left"
    )
    merged["correct"] = (merged["pred_class_simple"] == merged["true_class_simple"])

    # Summary metrics
    total = len(merged)
    nohit = int((merged["pred_class_simple"] == "NO_HIT").sum())  # unlikely; pred_class_simple could be NO_HIT
    acc = float(merged["correct"].mean()) if total else float("nan")

    print(f"[overlap] evaluated={total} | accuracy={acc:.3f}")
    # Per-class confusion on overlaps
    cm = pd.crosstab(merged["true_class_simple"], merged["pred_class_simple"], dropna=False)
    display(cm)

    # Any errors to inspect?
    wrong = merged[~merged["correct"]].sort_values(["true_class_simple","pred_class_simple","confidence"], ascending=[True, True, False])
    if not wrong.empty:
        print(f"[overlap] misclassified rows: {len(wrong)} (showing up to 10)")
        display(wrong.head(10))
    else:
        print("[overlap] No misclassifications among overlapped training IDs.")

    # Save detailed validation table
    out_csv = RESULTS / "training_overlap_validation.csv"
    merged.sort_values(["true_class_simple","correct","confidence"], ascending=[True, False, False]).to_csv(out_csv, index=False)
    print("[OK] wrote", out_csv)


In [ ]:
# === Build per-class MSAs from labeled sets in DIRS, then compute majority consensus for each class ===

def first_token(s): return (s or "").split()[0]

# Collect all per-class sequences (merge & dedup by ID)
class_to_records = {}
for d in DIRS:
    for ext in ("*.txt","*.fa","*.fasta","*.faa","*.fas"):
        for fp in Path(d).glob(ext):
            cls = fp.stem  # e.g., 8ca.alpha.45
            bucket = class_to_records.setdefault(cls, {})
            for r in SeqIO.parse(str(fp), "fasta"):
                rid = first_token(r.id or r.description)
                if rid not in bucket:
                    bucket[rid] = SeqRecord(Seq(str(r.seq)), id=rid, description="")

# Build per-class MSA with clustalo
def run(cmd): 
    return subprocess.run(shlex.split(cmd), check=True, capture_output=True, text=True)

for cls, id2rec in sorted(class_to_records.items()):
    recs = list(id2rec.values())
    if len(recs) < 2:
        continue
    in_fa  = ALIGN / f"{cls}.in.fasta"
    out_fa = ALIGN / f"{cls}.aln.fasta"
    SeqIO.write(recs, str(in_fa), "fasta")
    run(f'{clo} -i "{in_fa}" -o "{out_fa}" --force --threads=4 --seqtype=Protein --output-order=input-order')

# Majority-rule consensus
def majority_consensus_from_alignment(aln_fa: Path, threshold: float = 0.5) -> str:
    recs = list(SeqIO.parse(str(aln_fa), "fasta"))
    seqs = [str(r.seq) for r in recs]
    L = max(len(s) for s in seqs)
    cons = []
    for j in range(L):
        col = [s[j] if j < len(s) else "-" for s in seqs]
        letters = [c for c in col if c not in "-."]
        if not letters:
            cons.append("-"); continue
        vals, counts = np.unique(letters, return_counts=True)
        i = int(np.argmax(counts))
        frac = counts[i] / float(len(letters))
        cons.append(vals[i] if frac >= threshold else "X")
    return "".join(cons)

# Write consensus FASTA per class
consensus_paths = {}
for aln in sorted(ALIGN.glob("*.aln.fasta")):
    cls = aln.stem.replace(".aln","")  # e.g., 8ca.alpha.45
    cons = majority_consensus_from_alignment(aln, threshold=0.5)
    fa = CONS / f"{cls}.consensus.fasta"
    with open(fa, "w") as f:
        f.write(f">{cls}\n")
        for i in range(0, len(cons), 80):
            f.write(cons[i:i+80] + "\n")
    consensus_paths[cls] = fa

print(f"[OK] wrote {len(consensus_paths)} consensus sequences to {CONS}")


In [ ]:
# === Predict labels for 150k by aligning each query to every class consensus ===

# Input queries (150k)
QUERIES = REBASE / "tmp" / "uniprot_first_150000.faa"  

# Scoring for global alignment (simple, fast-ish)
MATCH, MISMATCH, GAP_OPEN, GAP_EXT = 1, -1, -5, -1

# Minimum identity to accept a class; otherwise NO_HIT
MIN_IDENTITY = 0.30    # 30% global identity to the consensus (tune as you like)

# Parallelism
N_PROC = max(1, (os.cpu_count() or 4) - 1)

# Load consensus sequences
consensus = []  # [(cls, simple_class, seqstr)]
def simple_class(cls):  # '8ca.alpha.45' -> 'alpha'
    import re
    m = re.match(r'^8ca\.([a-z]+)\.', cls)
    return m.group(1) if m else cls

for fa in sorted(CONS.glob("*.consensus.fasta")):
    cls = fa.stem.replace(".consensus","")
    seq = str(next(SeqIO.parse(str(fa), "fasta")).seq).replace("-", "")
    consensus.append((cls, simple_class(cls), seq))

assert consensus, "No consensus files found."

# Worker to classify one sequence
def classify_one(args):
    rec_id, seq = args
    best = None
    for cls, simp, cseq in consensus:
        # Global alignment to consensus
        # score_only speeds up; then compute identity from an explicit align once for the best
        alns = pairwise2.align.globalms(seq, cseq, MATCH, MISMATCH, GAP_OPEN, GAP_EXT, one_alignment_only=True)
        if not alns:
            continue
        score, a, b = alns[0].score, alns[0].seqA, alns[0].seqB
        # identity = matches / alignment length (excluding gap-gap)
        matches = sum(1 for x,y in zip(a,b) if x==y and x!='-' and y!='-')
        denom   = sum(1 for x,y in zip(a,b) if not (x=='-' and y=='-'))
        ident   = (matches/denom) if denom else 0.0
        if (best is None) or (ident > best["identity"]):
            best = {"pred_model": cls, "pred_class": simp, "identity": ident, "score": score}
    if best is None or best["identity"] < MIN_IDENTITY:
        return (rec_id, "NO_HIT", None, None, None)
    return (rec_id, best["pred_class"], best["pred_model"], best["identity"], best["score"])

# Stream queries and classify in parallel
def records_iter(path):
    for r in SeqIO.parse(str(path), "fasta"):
        yield (r.id.split()[0], str(r.seq))

rows = []
with mp.Pool(processes=N_PROC) as pool:
    for out in pool.imap_unordered(classify_one, records_iter(QUERIES), chunksize=64):
        rows.append(out)

df_pred = pd.DataFrame(rows, columns=["seq_id","pred_class","pred_model","identity","align_score"])
df_pred = df_pred.sort_values("seq_id")
out_csv = OUT / "consensus_identity_predictions_150k.csv"
df_pred.to_csv(out_csv, index=False)
print(f"[OK] wrote {len(df_pred):,} predictions → {out_csv}")


In [ ]:
# === Quick summaries ===

pred   = OUT / "consensus_identity_predictions_150k.csv"
df = pd.read_csv(pred)

print(df["pred_class"].value_counts(dropna=False))

# identity distribution by class
hits = df[df["pred_class"]!="NO_HIT"].copy()
plt.figure(figsize=(9,4))
for c in sorted(hits["pred_class"].unique()):
    x = hits.loc[hits["pred_class"]==c, "identity"].values
    x.sort()
    y = (np.arange(1,len(x)+1)/len(x))*100.0
    plt.step(x, y, where="post", label=c)
plt.xlabel("Global identity to class consensus")
plt.ylabel("Percent ≤ identity")
plt.title("ECDF of identity by predicted class (consensus-based)")
plt.legend()
plt.tight_layout()
plt.savefig(OUT / "identity_ecdf_by_class.png", dpi=150)
plt.show()



Each colored curve shows the ECDF of global percent identity between sequences assigned to that class and that class;s consensus

X-axis ==> global identity to class consensus 
similarity fraction used for assignment; higher = closer match

Y-axis  ==> percent of sequences =< that identity
 steeper a curve rise, the tighter the identity distribution 

The consensus-identity method found good matches only for a minority of sequences.

most sequences fall below the 30 % global identity threshold which means clustalo consensus alone doesn’t capture diversity

Among hits gamma > alpha > beta dominat showing these families are closer to training consensuses

The long right tails (up to ~0.6 identity) indicate some very high-confidence matches but most matches cluster around 0.35–0.45.

# Comparison

In [ ]:
# === Load Clustal-O based and HMMER based labels for comparison ===

# --- Paths ---
LABELS_HMMER   = RESULTS / "uniprot_first_150000_labels_with_confidence.csv"    #  HMMER based labels
LABELS_CLUSTER = OUT / "consensus_identity_predictions_150k.csv"               #  Clustal O consensus labels

# --- Load ---
df_hmm  = pd.read_csv(LABELS_HMMER)
df_cons = pd.read_csv(LABELS_CLUSTER)

print(f"[load] HMMER labels: {len(df_hmm):,}  |  ClustalΩ O labels: {len(df_cons):,}")

# Normalize IDs (same format)
df_hmm["seq_id"]  = df_hmm["seq_id"].astype(str).str.split().str[0]
df_cons["seq_id"] = df_cons["seq_id"].astype(str).str.split().str[0]

# Merge on sequence ID
df_merged = df_hmm.merge(
    df_cons[["seq_id","pred_class"]].rename(columns={"pred_class":"pred_class_consensus"}),
    on="seq_id", how="inner"
)
print(f"[merge] {len(df_merged):,} sequences appear in both sets")


In [ ]:

def class_counts(df, label_col, title, out_path):
    counts = (df[label_col]
              .value_counts(dropna=False)
              .rename_axis("class")
              .reset_index(name="count")
              .sort_values("count", ascending=False))
    plt.figure(figsize=(9,5))
    plt.bar(counts["class"], counts["count"])
    plt.title(title)
    plt.xlabel("Predicted class")
    plt.ylabel("Count")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.show()
    return counts

counts_hmm  = class_counts(df_hmm,  "pred_class", "HMMER-predicted Classes (150k)", RESULTS / "compare_hmmer_dist.png")
counts_cons = class_counts(df_cons, "pred_class", "Consensus-identity Predicted Classes (150k)", RESULTS / "compare_consensus_dist.png")

display(counts_hmm)
display(counts_cons)


In [ ]:


# === Compare overlap and agreement between classification methods ===
df_merged["agreement"] = (
    df_merged["pred_class"] == df_merged["pred_class_consensus"]
)

# Overall agreement rate (excluding NO_HIT on either side)
mask_valid = (df_merged["pred_class"] != "NO_HIT") & (df_merged["pred_class_consensus"] != "NO_HIT")
agree_rate = df_merged.loc[mask_valid, "agreement"].mean()
print(f"[agreement] {agree_rate:.3%} of classified sequences agree between methods "
      f"(excluding NO_HITs)")

# --- Confusion between methods ---
cm = pd.crosstab(df_merged["pred_class"], df_merged["pred_class_consensus"], dropna=False)
all_families = sorted(set(df_merged["pred_class"].unique()) | 
                     set(df_merged["pred_class_consensus"].unique()))
cm = cm.reindex(index=all_families, columns=all_families, fill_value=0)
print("[matrix] HMMER (rows) vs Consensus (columns)")

# Display table
display(cm)

# === NEW: Colored heatmap visualization ===
plt.figure(figsize=(8,6))
sns.heatmap(
    cm,
    annot=True, fmt="d", cmap="viridis", linewidths=0.5,
    cbar_kws={"label": "Count"}
)
plt.title("HMMER (rows) vs Consensus (columns) Classification Confusion Matrix")
plt.xlabel("Consensus-based Class")
plt.ylabel("HMMER-predicted Class")
plt.tight_layout()
plt.savefig(RESULTS / "hmmer_vs_consensus_confusion_heatmap.png", dpi=150)
plt.show()

# --- Top disagreement examples ---
disagree = df_merged[
    (df_merged["pred_class"] != df_merged["pred_class_consensus"]) &
    (df_merged["pred_class"] != "NO_HIT") &
    (df_merged["pred_class_consensus"] != "NO_HIT")
]




In [ ]:

# normalize ids
df_hmm["seq_id"]  = df_hmm["seq_id"].astype(str).str.split().str[0]
df_cons["seq_id"] = df_cons["seq_id"].astype(str).str.split().str[0]

# bring over consensus class + identity (+ score if present)
cols_to_merge = ["seq_id", "pred_class"]
if "identity" in df_cons.columns:
    df_cons = df_cons.rename(columns={"identity":"identity_consensus"})
    cols_to_merge.append("identity_consensus")
if "align_score" in df_cons.columns:
    df_cons = df_cons.rename(columns={"align_score":"align_score_consensus"})
    cols_to_merge.append("align_score_consensus")

df_merged = df_hmm.merge(df_cons[cols_to_merge], on="seq_id", how="inner") \
                  .rename(columns={"pred_class_y":"pred_class_consensus",
                                   "pred_class_x":"pred_class"})

print("[columns] merged has:", list(df_merged.columns))

In [ ]:


plt.figure(figsize=(10,5))
sns.violinplot(
    data=df_merged[df_merged["pred_class"]!="NO_HIT"],
    x="pred_class", y="confidence", inner="quartile", scale="width"
)
plt.title("Distribution of HMMER confidence per class")
plt.xlabel("Predicted class")
plt.ylabel("Confidence (−log10 E)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(RESULTS / "conf_per_class_violin.png", dpi=150)
plt.show()


In [ ]:
# === Extract NO_HIT sequences from 150k labeled set and write to FASTA ===

# Input files
LABELS_CSV = RESULTS / "uniprot_first_150000_labels_with_confidence.csv"
# Output
NO_HIT_FA = RESULTS / "no_hit_sequences_150k.fasta"

# Load labels and filter for NO_HIT
df = pd.read_csv(LABELS_CSV)
no_hit_ids = set(df[df["pred_class"] == "NO_HIT"]["seq_id"].astype(str).str.split().str[0])

print(f"[filter] Found {len(no_hit_ids):,} NO_HIT sequence IDs in labels")

# Extract sequences from original FASTA
no_hit_records = []
for rec in SeqIO.parse(str(QUERIES_FA), "fasta"):
    seq_id = rec.id.split()[0]
    if seq_id in no_hit_ids:
        # Keep original ID and description
        no_hit_records.append(rec)

print(f"[extract] Extracted {len(no_hit_records):,} sequences from FASTA")

# Write to output FASTA
if no_hit_records:
    SeqIO.write(no_hit_records, str(NO_HIT_FA), "fasta")
    print(f"[OK] Wrote {len(no_hit_records):,} NO_HIT sequences to: {NO_HIT_FA}")
    
    # Quick stats
    lengths = [len(r.seq) for r in no_hit_records]
    print(f"[stats] Length range: {min(lengths)}-{max(lengths)} aa")
    print(f"[stats] Mean length: {sum(lengths)/len(lengths):.1f} aa")
else:
    print("[warn] No sequences found to write")